In [1]:
import numpy as np

In [50]:
class Layer:
    def __init__(self):
        self.inputs = None
        self.outputs = None
    
    def forward(self,inp):
        pass
    
    def backward(self,output_gradient,lr):
        pass
        
    

In [93]:
class Dense(Layer):
    def __init__(self,input_size,output_size):
        self.w = np.random.randn(output_size, input_size)
        self.b = np.random.randn(output_size, 1)
    
    def forward(self,inp):
        self.inp = inp
        return np.dot(self.w,self.inp) + self.b
    
    def backward(self,output_gradient,lr):
        weight_gradient = output_gradient * self.inp.T
        input_gradient = np.dot(output_gradient,self.w.T)
        print(self.b)
        self.w -= weight_gradient * lr
        self.b -= output_gradient
        
        return input_gradient
        

In [52]:
class ActivationLayer(Layer):
    def __init__(self,activation,activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
        
    def forward(self,inp):
        self.inp = inp
        return self.activation(self.inp)
    
    def backward(self,output_gradient,lr):
        return np.multiply(output_gradient, self.activation_prime(self.inp))

In [53]:
class Tanh(ActivationLayer):
    def __init__(self):
        self.activation = lambda x : np.tanh(x)
        self.activation_prime = lambda x : 1 - (np.tanh(x)**2)
        super().__init__(self.activation,self.activation_prime)

In [54]:
def mse(y,y_pred):
    return np.mean(np.power(y - y_pred,2))
    
def mse_prime(y,y_pred):
    return (2/len(y)) * (y - y_pred)

In [55]:
def predict(network,inp):
    output = inp
    
    for layer in network:
        output = layer.forward(inp)
        
    return output
    

In [56]:
def train(network,X_train,y_train,loss,loss_prime,epochs=1000,lr=0.01):
    for _ in range(epochs):
        error = 0
        
        for x,y in zip(X_train,y_train):
            
            output = predict(network,x)
            
            error += loss(output,y)
            grad = loss_prime(output,y)
            
            for layer in reversed(network):
                grad = layer.backward(grad,lr)
            
        error /= len(X_train)

In [57]:
import sklearn
from sklearn import datasets

bc = datasets.load_breast_cancer()

X,y = bc.data,bc.target


from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [95]:
network = [
    Dense(X_train.shape[1], 10),
    Tanh(),
    Dense(10, 1),
    Tanh()
]

In [96]:
train(network,X_train, y_train, mse, mse_prime)

ValueError: shapes (1,10) and (30,) not aligned: 10 (dim 1) != 30 (dim 0)